In [107]:
import geopandas as gpd
from folium import (
    LayerControl,
    Element,
    TileLayer,
)
import plotly.express as px
from os import path



from core.downloads.geosampa import get_capabilities, get_features

# Promoção da Sustentabilidade Ambiental, Gestão de risco

## Formulário 7

O formulário associado a este notebook solicita os dados sobre 
`áreas de risco (hidrológico e deslizamentos)`. A justificativa para estes dados é a seguinte:

> As áreas de risco estão relacionadas à construção de moradias, em sua maioria em condições precárias, em locais com geológico-geotécnicas frágeis, não recomendadas para ocupação. O impacto de chuvas concentradas intensas, características de eventos climáticos extremos deve ser monitorado pelo Município, através de políticas públicas de gestão de risco e promoção da resiliência climática.

## Carregando as camadas de risco hidrológico e geológico

In [ ]:
get_capabilities('hidrológico')

In [ ]:
get_capabilities('deslizamento')

Como o formulário cita apenas a camada `proteção e defesa civil/mapeamento de areas de risco`, assumirei que a referência a deslizamentos seja sobre a camada de risco geológico.

In [ ]:
df_hid = get_features('geoportal:risco_hidrologico')
df_hid.head()

In [ ]:
df_geo = get_features('geoportal:area_risco_geologico')
df_geo.head()

Além dos dados de riscos hidrogeológicos, também precisaremos dos dados do Censo de 2022 para a estimativa populacional. Os dados básicos, como número de domicílios e população, são disponibilizados diretamente no geopackage com as geometrias de setores censitários.

In [ ]:
df_censo = gpd.read_file('https://ftp.ibge.gov.br/Censos/Censo_Demografico_2022/Agregados_por_Setores_Censitarios/malha_com_atributos/setores/gpkg/UF/SP/SP_setores_CD2022.gpkg')
df_censo.head()

In [ ]:
df_censo = df_censo.loc[df_censo['CD_MUN']=='3550308']
df_censo.head()

## Carregando a camada de cobertura vegetal

In [ ]:
get_capabilities('veg')

In [ ]:
df_veg = get_features('geoportal:cobertura_vegetal')
df_veg.head()


## Carregando a camada de quadras viárias

In [ ]:
get_capabilities('viaria')

In [ ]:
df_quadras = get_features('geoportal:quadra_viaria_editada')
df_quadras.head()


## Ajustando as projeções

Vamos revisar os sistemas de coordenadas de todos os geodataframes para garantir que estão na mesma projeção.

In [ ]:
for gdf in [df_geo, df_hid, df_censo, df_veg, df_quadras]:
    print(gdf.columns[:5])
    print(gdf.crs)

Como o geodataframe do censo está em outro crs, precisamos convertê-lo para o `epsg:31983`.

In [13]:
df_censo = df_censo.to_crs('EPSG:31983')

# Calculando a população de cada área de risco

Primeiro, vamos inspecionar visualmente os geodataframes.

In [ ]:
df_geo.explore()

In [ ]:
df_censo.iloc[:500].explore()

In [ ]:
df_veg.iloc[:500].explore()

## Calculando as interseções

1. Remover as áreas de vegetação dos setores censitarios;
1. Calcular a interseção com as quadras viárias (para remover as ruas);
1. Calcular a área de cada setor censitário ajustado (nova área total);
1. Calcular a interseção com as áreas de risco;
1. Calcular o peso da interseção em relação ao setor ajustado;
1. Calcular a população e moradias com base no peso;
1. Agregar população e moradia por área de risco;

In [ ]:
df_censo['NM_DIST'].value_counts()

### Testando com o distrito de Grajaú

#### Filtrando os geodataframes

In [ ]:
df_censo_filtrado = df_censo[df_censo['NM_DIST']=='Grajaú']
df_censo_filtrado

Primeiro, vamos filtrar a área de vegetação para os tipos mais adequados para o ajuste dos setores censitários.

In [ ]:
df_veg_filtrada = df_veg[df_veg.intersects(df_censo_filtrado.union_all())]

df_veg_filtrada

#### Removendo a vegetação

Depois, calculamos a diferença entre os setores censitários e a vegetação, para remover as área não povoadas.

In [ ]:
ol1 = gpd.overlay(df_censo_filtrado, df_veg_filtrada,
                  how='difference',
                  keep_geom_type=True)

ol1.head()

In [22]:
# tooltip_columns = ['CD_SETOR', 'v0001', 'v0002', 'v0003', 'v0004', 'v0005', 'v0006', 'v0007']

# m = df_censo_filtrado.explore(
#     tiles='CartoDB positron',
#     color='blue',  # Cor dos setores censitários
#     style_kwds={'fillOpacity': 0.2},  # Transparência do preenchimento
#     name='Setores Censitários Originais',
#     tooltip=tooltip_columns
# )
# m = ol1.explore(
#     m=m,
#     color='purple',  # Cor das interseções
#     style_kwds={'fillOpacity': 0.5},  # Transparência do preenchimento
#     name='Setores Censitários Ajustados',
#     tooltip=tooltip_columns
# )

# # Adicionar camada de satélite
# TileLayer(
#     tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
#     attr='Google Satellite',
#     name='Google Satellite'
# ).add_to(m)

# # Adicionar controle de camadas
# LayerControl().add_to(m)

# # Criar a legenda personalizada
# legend_html = """
# <div style="position: fixed; 
#             bottom: 50px; left: 50px; width: 200px; height: 120px; 
#             background-color: white; z-index:1000; padding: 10px; 
#             border: 2px solid grey; border-radius: 5px;">
#     <h4>Legenda</h4>
#     <i style="background: blue; width: 10px; height: 10px; display: inline-block;"></i> Setores Censitários Originais<br>
#     <i style="background: purple; width: 10px; height: 10px; display: inline-block;"></i> Setores Censitários Ajustados<br>
# </div>
# """
# m.get_root().html.add_child(Element(legend_html))

# m

#### Removendo a malha viária

Depois, removemos a malha viária dos polígonos calculando a interseção com as quadras viárias.

In [ ]:
ol2 = gpd.overlay(ol1, df_quadras[['geometry']],
            how='intersection',
            keep_geom_type=True)
ol2

In [ ]:
ol2.iloc[:500].explore()

Como nosso interesse é pelo setor censitário, vamos agrupar as interseções por setor censitário. Além disso, precisaremos da área do setor ajustado para os cálculos de ponderação, vamos adicionar a coluna agora.

In [ ]:
ol2 = (
    ol2
    .dissolve(by='CD_SETOR', aggfunc='first')
    .reset_index()
)

ol2['area_setor_ajustado'] = ol2.area
ol2

#### Calculando a interseção com as áreas de risco

Depois, precisaremos calcular a interseção de cada um dos dataframes de risco com os setores censitários ajustados.

In [ ]:
ol3 = gpd.overlay(
    ol2.loc[:, ['CD_SETOR', 'v0001', 'v0002', 'v0003', 'v0004',
                'v0005', 'v0006', 'v0007', 'area_setor_ajustado', 'geometry']],
    df_geo,
    how='intersection',
    keep_geom_type=True)

ol3.head()

Vamos avaliar visualmente o resultado da interseção com base na primeira área de risco.

In [ ]:
id_area = ol3.loc[:, 'id'].iloc[0]
id_area

In [ ]:
cd_setor_list = ol3.loc[ol3['id']==id_area, 'CD_SETOR'].tolist()
ol3.loc[ol3['id']==id_area, ['id', 'CD_SETOR']]

In [ ]:
m = df_censo[df_censo['CD_SETOR'].isin(
    cd_setor_list)].explore(name='setor censitário', color='navy')

m = ol2[ol2['CD_SETOR'].isin(cd_setor_list)].explore(
    m = m, name='setor censitário ajustado')

m = df_geo[df_geo['id'] == id_area].explore(
    m=m, color='orange', name='area de risco')

filtered_ol3 = ol3.loc[ol3['id'] == id_area, [
    'id', 'CD_SETOR', 'geometry']]
m = filtered_ol3.explore(m=m, color='purple', name='interseção')

LayerControl().add_to(m)

m

Na inspeção visual, nota-se que algumas interseções não aparentam representar áreas com moradias, mas simplesmente leves discrepâncias no desenho dos polígonos sobre áreas não populadas (ruas,  canteiros, etc.).

Uma opção de passo adicional seria explodir todos os polígonos e aplicar um buffer negativo para que esses recortes sem significado prático fossem removidos. Porém, como as áreas relativas a essas áreas também devem causar pouca distorção nos valores, não realizaremos essa remoção neste momento.

Antes de passar para os cálculos, vamos adicionar uma coluna com a área de cada interseção ao geodataframe.

In [ ]:
ol3['area_intersecao'] = ol3['geometry'].area
ol3

#### Calculando a proporção das áreas

O primeiro passo é calcular a proporção entre a área da interseção e a área do setor censitário.

In [ ]:
ol3['prop_setor'] = ol3['area_intersecao']/ol3['area_setor_ajustado']
ol3.head()

Sabendo que a proporção da interseção não deve ser maior do que 1, vamos conferir os valores de interseção.

In [ ]:
ol3['prop_setor'].hist()

Como a distribuição apresenta muitos valores abaixo de 0.1, vamos analisar esses casos visualmente para confirmar que fazem sentido.

In [79]:
filtro_intersecoes_pequenas = ol3['prop_setor']<=0.1
setores_02 = ol3.loc[filtro_intersecoes_pequenas, 'CD_SETOR'].tolist()
areas_risco_02 = ol3.loc[filtro_intersecoes_pequenas, 'id'].tolist()

In [ ]:
# Criar o mapa com as camadas
m = df_censo[df_censo['CD_SETOR'].isin(setores_02)].explore(name='Setor Censitário')

m = df_geo[df_geo['id'].isin(areas_risco_02)].explore(
    m=m, color='orange', name='Área de Risco')

filtered_ol3 = ol3.loc[filtro_intersecoes_pequenas, ['id', 'CD_SETOR', 'geometry', 'prop_setor']]
m = filtered_ol3.explore(m=m, color='purple', name='Interseção')

LayerControl().add_to(m)

# Adicionar a legenda personalizada
legend_html = """
<div style="position: fixed; 
            bottom: 50px; left: 50px; width: 200px; height: 120px; 
            background-color: white; z-index:1000; padding: 10px; 
            border: 2px solid grey; border-radius: 5px;">
    <h4>Legenda</h4>
    <i style="background: orange; width: 10px; height: 10px; display: inline-block;"></i> Área de Risco<br>
    <i style="background: purple; width: 10px; height: 10px; display: inline-block;"></i> Interseção<br>
    <i style="background: blue; width: 10px; height: 10px; display: inline-block;"></i> Setor Censitário
</div>
"""
m.get_root().html.add_child(Element(legend_html))

m

A inspeção visual mostra que parte das interseções não aparentam ser significativas, mas não causarão distorção relevante no cálculo, devido ao fator de pontederação baixo. Outras, entretanto, aparentam ser significativas, mas se tratam de interseções pequenas em comparação com os setores censitários, ocorrendo principalmente em setores sensitários maiores.

#### Calculando a população

O primeiro passo é calcular a população correspondente a cada interseção.

In [ ]:
ol3['pop_estimada'] = ol3['v0001']*ol3['prop_setor']

(
    ol3
    .loc[:, ['id','CD_SETOR','v0001', 'prop_setor', 'pop_estimada']]
    .sort_values('pop_estimada', ascending=False)
)

Depois, calculamos a população agregada por área de risco.

In [ ]:
area_risco_pop = (
    ol3
    .loc[:, ['id', 'pop_estimada']]
    .groupby('id')
    .sum()
    .round(0)
    .reset_index()
)

area_risco_pop['pop_estimada'] = area_risco_pop['pop_estimada'].astype(int)
area_risco_pop

Finalmente, trazemos a coluna de população para o dataframe original.

In [ ]:
df_geo_pop = df_geo.merge(
    area_risco_pop,
    how='inner'
)

df_geo_pop.head()

#### Visualizando a população

Para visualizar o resultado, vamos criar algumas visualizações. Primeiro, vamos criar dois *treemaps* por tipo de processo e grau de risco, alterando a ordem da hierarquia.

In [ ]:
df_tree = (
    df_geo_pop
    .groupby(['tx_tipo_processo_geologico', 'tx_grau_de_risco_geologico'], as_index=False)
    ['pop_estimada']
    .sum()
)

df_tree

In [ ]:
# Criar o gráfico do tipo treeview
fig = px.treemap(
    df_tree,
    path=['tx_grau_de_risco_geologico', 'tx_tipo_processo_geologico'],  # Caminho hierárquico
    values='pop_estimada',               # Valores para o tamanho das caixas
    title='Somatório de População Estimada por Tipo de Processo Geológico'
)

# Exibir o gráfico
fig.show()

In [ ]:
# Criar o gráfico do tipo treeview
fig = px.treemap(
    df_tree,
    path=['tx_tipo_processo_geologico', 'tx_grau_de_risco_geologico'],  # Caminho hierárquico
    values='pop_estimada',               # Valores para o tamanho das caixas
    title='Somatório de População Estimada por Tipo de Processo Geológico'
)

# Exibir o gráfico
fig.show()

Por último, vamos criar um mapa coroplético das áreas de risco, adicionando as camadas de setores censitários envolvidos, interseções envolvidas (e população atribuida) e adicionar também uma camada de satelite.

In [ ]:

# Filtrar os setores censitários presentes em ol3
setores_presentes = df_censo[df_censo['CD_SETOR'].isin(ol3['CD_SETOR'].unique())]
# Adicionar os setores censitários ao mapa com apenas o contorno
m = setores_presentes.explore(
    color=None,  # Remove o preenchimento
    style_kwds={'fill': False, 'color': 'skyblue'},  # Define apenas o contorno azul
    name='Setores Censitários',
    tiles='CartoDB positron'  # Mapa base
)

# Criar o mapa base
m = df_geo_pop.explore(
    m=m,
    column='pop_estimada',  # Coluna para o mapa coroplético
    cmap='YlOrRd',          # Paleta de cores
    legend=True,            # Adicionar legenda
    legend_kwds={
        'caption': 'População total',
        'position': 'bottomright',  # Alterar posição da legenda
        'frameon': True             # Adicionar quadro de fundo
    },
    tooltip=['tx_grau_de_risco_geologico', 'tx_tipo_processo_geologico', 'pop_estimada'],  # Informações no tooltip
    name='Áreas de Risco'   # Nome da camada
)

# Adicionar as interseções ao mapa
m = ol3.explore(
    m=m,
    column='pop_estimada',  # Coluna para o mapa coroplético
    cmap='YlOrRd',          # Paleta de cores
    legend=True,            # Adicionar legenda
    legend_kwds={
        'caption': 'População parcial',
        'position': 'bottomright',  # Alterar posição da legenda
        'frameon': True             # Adicionar quadro de fundo
    },
    tooltip=['CD_SETOR', 'pop_estimada'],  # Informações no tooltip
    name='Interseções'      # Nome da camada
)

# Adicionar outra camada base (Google Satellite)
TileLayer(
    tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='Google Satellite',
    name='Google Satellite'
).add_to(m)

# Adicionar controle de camadas
LayerControl(position='bottomleft').add_to(m)

# Exibir o mapa
m


In [42]:
m.save('plots/população em áreas de risco.html')

# Generalizando o cálculo para toda a cidade

Primeiro, vamos criar uma função genérica para fazer a interpolação da variável entre dois geodataframes.

In [95]:

def areal_weighted_interpolation(
        left: gpd.GeoDataFrame,
        right: gpd.GeoDataFrame,
        right_id_col: str,
        original_var_name: str,
        final_var_name: str = None):
    """
    Perform areal weighted interpolation between two GeoDataFrames.
    This function calculates the weighted interpolation of a variable from one GeoDataFrame (`left`) 
    to another GeoDataFrame (`right`) based on the proportion of intersection areas. It is commonly 
    used in spatial analysis to transfer data between different spatial units.
    Parameters:
    -----------
    left : gpd.GeoDataFrame
    The source GeoDataFrame containing the original spatial data and variable to be interpolated.
    right : gpd.GeoDataFrame
    The target GeoDataFrame to which the interpolated variable will be transferred.
    right_id_col : str
    The column name in the `right` GeoDataFrame that uniquely identifies spatial units.
    original_var_name : str
    The column name in the `left` GeoDataFrame containing the variable to be interpolated.
    final_var_name : str, optional
    The column name for the interpolated variable in the resulting GeoDataFrame. 
    If not provided, the `original_var_name` will be used.
    Returns:
    --------
    final_gdf : gpd.GeoDataFrame
    A GeoDataFrame containing the `right` GeoDataFrame with the interpolated variable added.
    Notes:
    ------
    - The function assumes that both GeoDataFrames use the same coordinate reference system (CRS).
    - The intersection areas are calculated using the `gpd.overlay` method with `how='intersection'`.
    - The interpolated variable is calculated as the product of the original variable and the 
        proportion of intersection area relative to the total area of the `left` GeoDataFrame.
    Example:
    --------
    >>> interpolated_gdf = areal_weighted_interpolation(
        left=source_gdf,
        right=target_gdf,
        right_id_col='region_id',
        original_var_name='population',
        final_var_name='interpolated_population'
    >>> interpolated_gdf.head()
    """

    left = left.copy()
    right = right.copy()

    left['total_area'] = left.area

    inter = gpd.overlay(
        left,
        right,
        how='intersection',
        keep_geom_type=True)

    inter['intersection_area'] = inter.area

    inter['inter_prop'] = inter['intersection_area']/inter['total_area']

    if not final_var_name:
        final_var_name = original_var_name

    inter[final_var_name] = inter[original_var_name]*inter['inter_prop']

    right_interpolated = (
        inter
        .loc[:, [right_id_col, final_var_name]]
        .groupby(right_id_col)
        .sum()
        .round(0)
        .reset_index()
    )

    right_interpolated[final_var_name] = right_interpolated[final_var_name].astype(int)
    right_interpolated

    final_gdf = right.merge(
        right_interpolated,
        how='inner',
        on=right_id_col
    )
    
    return final_gdf

Agora, preparamos uma função para calcular os polígonos ajusados de todos os setores censitários da cidade.

In [98]:
def prepare_tracts():

    ol1 = gpd.overlay(df_censo, df_veg,
                    how='difference',
                    keep_geom_type=True)

    ol2 = (
        gpd.overlay(ol1, df_quadras[['geometry']],
                    how='intersection',
                    keep_geom_type=True)
            .dissolve(by='CD_SETOR', aggfunc='first')
            .reset_index()
    )

    ol2['area_setor_ajustado'] = ol2.area
    
    return ol2

## Preparando os setores censitários

In [ ]:
df_censo_ajustado = prepare_tracts()
df_censo_ajustado.head()

## Calculando a população para risco hidrológico

Com os setores censitários ajustados, vamos calcular a população para as áreas de risco hidrológico.

In [ ]:
df_hid.columns

In [ ]:
df_hid_pop = areal_weighted_interpolation(
    df_censo_ajustado,
    df_hid,
    'id',
    'v0001',
    'populacao_estimada'
)

df_hid_pop.head()

In [ ]:
df_hid_final = areal_weighted_interpolation(
    df_censo_ajustado,
    df_hid_pop,
    'id',
    'v0002',
    'total_domicilios_estimado'
)

df_hid_final.head()

## Calculando a população para risco geológico

Com os setores censitários ajustados, vamos calcular a população para as áreas de risco geológico.

In [ ]:
df_geo.columns

In [ ]:
df_geo_pop = areal_weighted_interpolation(
    df_censo_ajustado,
    df_geo,
    'id',
    'v0001',
    'populacao_estimada'
)

df_geo_pop.head()

In [ ]:
df_geo_final = areal_weighted_interpolation(
    df_censo_ajustado,
    df_geo_pop,
    'id',
    'v0002',
    'total_domicilios_estimado'
)

df_geo_final.head()

# Exportando os arquivos

Finalmente, salvamos os arquivos com as camadas com os totais de população e domicílios.

In [114]:
output_dir = path.join('data_output', 'urbanismo')

for c, gdf in [('risco_hidrologico', df_hid_final),
               ('risco_geologico', df_geo_final)]:
    filename=path.join(output_dir, c)
    gdf.to_file(f'{filename}.gpkg', driver='GPKG', layer=c)